In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jenil\\OneDrive\\Desktop\\projects\\vehicle-classifier\\vehicle-classification-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\jenil\\OneDrive\\Desktop\\projects\\vehicle-classifier\\vehicle-classification-project'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# entity

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

# configuration

In [8]:
from vehicleclassifier.constants import *
from vehicleclassifier.utils.common import create_directories,read_yaml,save_json


In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="dataset\\test",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# components

In [10]:
from urllib.parse import  urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.9
        )
        # print("==================",os.listdir(self.config.training_data))
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-06-11 12:37:00,773: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-11 12:37:00,775: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-11 12:37:00,776: INFO: common: created directory at: artifacts]
Found 636 images belonging to 5 classes.
20/20 [==============================] - 24s 593ms/step - loss: 4.8985 - accuracy: 0.7767
[2023-06-11 12:37:25,298: INFO: common: json file saved at: scores.json]


In [ ]:
import numpy as np
from PIL import Image

def predict(image_path,model_path):
    model=tf.keras.models.load_model(model_path)
    image = tf.keras.utils.load_img(image_path,
                                    target_size=(224,224),
                                    interpolation='bilinear')
    # image = Image.open(image_path)
    # resized = image.resize((224,224),resample=Image.BILINEAR)
    image_array = np.array(image)
    normalised = image_array *1./255
    normalised = normalised[np.newaxis,:,:]
    prediction = model.predict(normalised)
    print(prediction[0])

In [ ]:
predict('dataset\\test\\truck\\truck0.png','artifacts\\training\\model.h5')